In [1]:
# *** INITIALISATION ***#

import os
from pathlib import Path
from dotenv import load_dotenv
import datetime

load_dotenv()

# Timestamp
date_now = datetime.date.today().isoformat()
datetime_now = datetime.datetime.now().isoformat()

In [ ]:
# *** TÉLÉCHARGEMENT ***#

import pandas as pd
from requests import get

# decp_augmente_valides_file: Path = Path(f'data/decp_augmente_valides_{date_now}.csv')
decp_augmente_valides_file: Path = Path(f"data/decp_augmente_valides_2024-04-12.csv")


if not (os.path.exists(decp_augmente_valides_file)):
    request = get(os.getenv("DECP_ENRICHIES_VALIDES_URL"))
    with open(decp_augmente_valides_file, "wb") as file:
        file.write(request.content)
else:
    print("DECP d'aujourd'hui déjà téléchargées")

df: pd.DataFrame = pd.read_csv(
    decp_augmente_valides_file,
    sep=";",
    dtype="object",
    index_col=None,
)

In [ ]:
from numpy import NaN

df.replace([NaN, None], "", inplace=True, regex=False)
# df[['id', 'datePublicationDonnees']].loc[df['datePublicationDonnees'].str.contains("September")]

In [ ]:
# *** REDRESSEMENT ***#

# Suppression des lignes en doublon
# Exemple : 20005584600014157140791205100

df = df.drop_duplicates(ignore_index=True)

# Dates

columns_date = ["datePublicationDonnees", "dateNotification"]

date_replacements = {
    # ID marché invalide et SIRET de l'acheteur
    "0002-11-30": "",
    "September, 16 2021 00:00:00": "2021-09-16",  # 20007695800012 19830766200017 (plein !)
    "16 2021 00:00:00": "",
    "0222-04-29": "2022-04-29",  # 202201L0100
    "0021-12-05": "2022-12-05",  # 20222022/1400
    "0001-06-21": "",  # 0000000000000000 21850109600018
    "0019-10-18": "",  # 0000000000000000 34857909500012
    "5021-02-18": "2021-02-18",  # 20213051200 21590015000016
    "2921-11-19": "",  # 20220057201 20005226400013
    "0022-04-29": "2022-04-29",  # 2022AOO-GASL0100 25640454200035
}

for col in columns_date:
    df[col] = df[col].replace(date_replacements, regex=False)


# Nombres

df["dureeMois"] = df["dureeMois"].replace("", NaN)
df["montant"] = df["montant"].replace("", NaN)

# Identifiants de marchés

id_replacements = {"[,\./]": "_"}

df["id"] = df["id"].replace(id_replacements, regex=True)

# Nature

nature_replacements = {"Marche": "Marché", "subsequent": "subséquent"}

df["nature"] = df["nature"].str.capitalize()
df["nature"] = df["nature"].replace(nature_replacements, regex=True)

In [ ]:
# ***  TYPES DE DONNÉES ***#

numeric_dtypes = {
    "dureeMois": "Int64",  # contrairement à int64, Int64 autorise les valeurs nulles https://pandas.pydata.org/docs/user_guide/integer_na.html
    "montant": "float64",
}

for column in numeric_dtypes:
    df[column] = df[column].astype(numeric_dtypes[column])


date_dtypes = ["datePublicationDonnees", "dateNotification"]

for column in date_dtypes:
    df[column] = pd.to_datetime(df[column], format="mixed", dayfirst=True)

In [ ]:
# ***  ANALYSES AVANCÉES ***#

# Les sources d'où proviennent les données
sources = df["source"].unique()

for source in sources:
    print(
        f"""
# {source}
Nombre de marchés : {df[["source"]].loc[df["source"]==source].index.size}
Nombre d'acheteurs uniques : {len(df[["acheteur.id", "source"]].loc[df["source"]==source]['acheteur.id'].unique())}
"""
    )

In [ ]:
# VERS LE FORMAT DECP-TABLE-SCHEMA #

# Explosion des champs titulaires sur plusieurs lignes

df["titulaire.id"] = [[] for r in range(len(df))]
df["titulaire.denominationSociale"] = [[] for r in range(len(df))]
df["titulaire.typeIdentifiant"] = [[] for r in range(len(df))]

for num in range(1, 4):
    mask = df[f"titulaire_id_{num}"] != ""
    df.loc[mask, "titulaire.id"] += df.loc[mask, f"titulaire_id_{num}"].apply(
        lambda x: [x]
    )
    df.loc[mask, "titulaire.denominationSociale"] += df.loc[
        mask, f"titulaire_denominationSociale_{num}"
    ].apply(lambda x: [x])
    df.loc[mask, "titulaire.typeIdentifiant"] += df.loc[
        mask, f"titulaire_typeIdentifiant_{num}"
    ].apply(lambda x: [x])

df = df.explode(
    ["titulaire.id", "titulaire.denominationSociale", "titulaire.typeIdentifiant"],
    ignore_index=True,
)

In [ ]:
# Ajout colonnes manquantes

df["uid"] = df["acheteur.id"] + df["id"]
df["donneesActuelles"] = ""  # TODO
df["anomalies"] = ""  # TODO

In [ ]:
# *** ENREGISTREMENT AU FORMAT CSV ET PARQUET ***#

import shutil

distdir = Path("./dist")
if os.path.exists(distdir):
    shutil.rmtree("dist")
os.mkdir("dist")
os.chdir(distdir)

# CSV

# Schéma cible
df_standard = pd.read_csv(
    "https://raw.githubusercontent.com/ColinMaudry/decp-table-schema/main/exemples/exemple-valide.csv",
    index_col=None,
)

df = df[df_standard.columns]
df.to_csv("decp.csv", index=None)
df.to_parquet("decp.parquet", index=None)


# Sans titulaires (1 ligne par marché)
df_sans_titulaires = df.drop(
    columns=[
        "titulaire.id",
        "titulaire.denominationSociale",
        "titulaire.typeIdentifiant",
    ]
)
df_sans_titulaires = df_sans_titulaires.drop_duplicates(ignore_index=True)
df_sans_titulaires.to_csv("decp-sans-titulaires.csv", index=None)
df_sans_titulaires.to_parquet("decp-sans-titulaires.parquet", index=None)